In [ ]:
# Import necessary libraries
import pandas as pd
import requests
import json

In [ ]:
#AUTOMATION AREA
# 1) Get company tickers to format URL string
# 2) Format string as such: "https://financialmodelingprep.com/api/v3/income-statement/" + companyName + "?datatype=csv&apikey=" + API_KEY

In [ ]:
keyFile="/home/wintergreen/Projects/fskeyfile.txt"
with open(keyFile) as f:
    key = str(f.readline().replace("\n",""))

In [ ]:
API_KEY = key

In [ ]:
TICKER_TARGET = "AAPL"

In [ ]:
URL = "https://financialmodelingprep.com/api/v3/income-statement/" + TICKER_TARGET + "?limit=120&apikey=" + API_KEY

In [ ]:
r = requests.get(URL)

In [ ]:
r = r.json()

In [ ]:
pd.DataFrame.from_dict(r)